## Firm-level Climate Change Exposure: Vilkov, G. et al. (2020)

复现：伊日敏（武汉大学）

### **算法原理**

以下是该论文中“关键词发现算法”的完整流程。

这个算法在方法论上的核心贡献是，它并非使用一个静态的、预先定义的词典来识别文本，而是采用一种自下而上的、基于机器学习的方法，**从文本本身发现与特定主题相关的、不断演化的“行话”（jargon）**。

---

#### **阶段一：数据准备与集合定义 (Data Preparation & Set Definition)**

此阶段的目标是为后续的监督学习模型构建一个高质量的、带有明确标签的训练集。

1.  [cite_start]**初始种子集 ($C^0$)**: 算法的起点是一个人工预先定义的、包含50个高置信度（high-confidence）二元词组（bigrams）的列表 [cite: 1333, 1349][cite_start]。**这些词组是与“气候变化”主题明确相关的**，例如 "climate change"、"global warming" 和 "carbon emission" [cite: 1349]。这个集合构成了算法的“锚点”。

2.  [cite_start]**参照集 R (Reference Set)**: 基于种子集 $C^0$，算法从全部电话会议文本（corpus）中，筛选出所有包含这些种子词组的句子，形成参照集R [cite: 1350]。
    * [cite_start]**作用**: 这个集合被视为后续监督学习的**正样本（positive examples）**。它包含了约6万个句子，被认为是“几乎可以肯定与气候变化讨论相关”的文本 [cite: 1350]。

3.  [cite_start]**搜索集 S (Search Set)**: 语料库中所有**不**包含 $C^0$ 中任何词组的句子，被归入搜索集S [cite: 1350]。
    * [cite_start]**作用**: 这个集合极其庞大（约7000万个句子），包含了大量与主题无关的“噪音”，但也可能隐藏着我们想要发现的、未使用种子词汇的“气候变化”相关句子 [cite: 1351]。它构成了算法待探索的广阔空间。

#### **阶段二：监督学习与句子分类 (Supervised Learning & Sentence Classification)**

此阶段的目标是利用阶段一准备好的数据，训练一个分类器，用以在庞大的搜索集S中识别出更多潜在相关的句子。

1.  [cite_start]**构建训练集**: 算法通过组合以下两部分来构建训练集 [cite: 1352]：
    * **正样本**: 全部来自参照集R的句子。
    * **负样本**: 从搜索集S中随机抽取的10万个句子。这个设计的逻辑是，由于S集极其庞大，随机抽样出的句子绝大多数与气候变化无关，因此可以作为**负样本**的有效代理（proxy）。

2.  **模型训练与调优**:
    * [cite_start]**分类器选择**: 作者使用了三种不同的机器学习分类器进行训练：多项式朴素贝叶斯（Multinomial Naive Bayes）、支持向量机（Support Vector Classification）和随机森林（Random Forest） [cite: 1353]。
    * [cite_start]**超参数调优**: 为了让每个分类器达到最佳性能，算法采用了**网格搜索交叉验证 (grid-search cross-validation)** 的方法来选择最优的超参数组合 [cite: 1354]。

3.  **预测与目标集生成**:
    * [cite_start]**概率预测**: 三个经过调优的分类器被用于预测搜索集S中每一个句子的“属于R的概率” [cite: 1358]。
    * [cite_start]**生成目标集 T (Target Set)**: 对于S中的一个句子，只要**有任何一个**分类器给出的预测概率**高于0.8**，该句子就会被归入一个新的集合，称为目标集T [cite: 1358]。
    * [cite_start]**结果**: 这个目标集T包含了约70万个句子，它们虽然不含初始的种子词组，但被模型高度怀疑是与气候变化相关的文本 [cite: 1359]。

#### **阶段三：关键词发现与度量构建 (Keyword Discovery & Measure Construction)**

此阶段是算法的核心，目标是从上一步筛选出的目标集T中，反向工程挖掘出新的、有意义的二元词组，并最终构建量化指标。

1.  [cite_start]**关键词辨别 (Discrimination)**: 算法的核心发现步骤是识别那些能够最好地区分目标集T和非目标集 (S\T) 的二元词组 [cite: 1360]。
    * [cite_start]**频率比较**: 对于同时出现在T和S\T中的词组，算法会计算它们在两个集合中的文档频率（document frequencies），并保留那些在T中出现得更频繁的词组 [cite: 1363, 1364]。

2.  **统计排序与筛选**:
    * [cite_start]**似然度量**: 算法使用了一个修改版的似然度量（likelihood metric）来对上一步筛选出的候选词组进行排序，以评估其区分能力 [cite: 1366][cite_start]。由于搜索集S规模巨大，原文作者使用对数伽马函数（log-gamma function）代替了原方法中的伽马函数以保证计算可行性 [cite: 1367]。
    * [cite_start]**筛选新词组 ($C^S$)**: 在所有候选词组中，似然度排名前5%的词组被最终选定，形成一个新的“已发现词组集”$C^S$，这个集合包含了约5000个新的二元词组 [cite: 1366]。

3.  **构建最终词库与度量**:
    * [cite_start]**最终词库 C**: 最终的气候变化二元词组库 C，是**初始种子集 $C^0$** 和**新发现词组集 $C^S$** 的并集 ($C = C^0 \cup C^S$) [cite: 1369]。
    * [cite_start]**构建度量指标 (CCExposure)**: 基于最终词库C，为每一份电话会议记录计算一个“气候变化风险敞口”分数。其计算方法为：该记录中属于词库C的二元词组总数，除以该记录中总的二元词组数 [cite: 390, 391, 395][cite_start]。这个过程同样被应用于构建三个子主题（机会、监管和物理风险）的度量指标 [cite: 373, 382, 399, 400]。

通过以上三个阶段，该研究成功地构建了一个动态的、能自我迭代的关键词发现与文本度量框架。